# Survey Distribution & Collection Tool

In [18]:
import os
import boto3
import xml.etree.ElementTree as ET
from botocore.config import Config

os.environ['AWS_ACCESS_KEY_ID']     = 
os.environ['AWS_SECRET_ACCESS_KEY'] = 
os.environ['AWS_REGION']            = 

config = Config(region_name='us-east-1', signature_version='v4')
mturk = boto3.client(
    'mturk',
    config=config,
    aws_access_key_id=os.environ['AWS_ACCESS_KEY_ID'],
    aws_secret_access_key=os.environ['AWS_SECRET_ACCESS_KEY'],
    endpoint_url='https://mturk-requester-sandbox.us-east-1.amazonaws.com'
)

In [20]:
def create_survey_hit(survey_link: str,
                      reward: str = "0.01",
                      max_assignments: int = 1) -> str:
    xml_question = f"""<?xml version="1.0" encoding="UTF-8"?>
<ExternalQuestion xmlns="http://mechanicalturk.amazonaws.com/AWSMechanicalTurkDataSchemas/2006-07-14/ExternalQuestion.xsd">
  <ExternalURL>{survey_link}</ExternalURL>
  <FrameHeight>600</FrameHeight>
</ExternalQuestion>"""
    resp = mturk.create_hit(
        Title="Sandbox Survey Test",
        Description="请在此 HIT 中打开问卷并填写",
        Reward=reward,
        MaxAssignments=max_assignments,
        LifetimeInSeconds=3600,
        AssignmentDurationInSeconds=1800,
        Question=xml_question
    )
    hit_id = resp['HIT']['HITId']
    print(f"Created HIT: {hit_id}")
    return hit_id

survey_link = "https://qualtricsxmnznfsjncm.qualtrics.com/jfe/form/SV_00tpW4sa8vcRYma"
hit_id = create_survey_hit(survey_link)

Created HIT: 34XASH8KMGJGNH2G1AZWS8W5TGIMP1


In [22]:
print("\n👉 请在 https://workersandbox.mturk.com/ 用 Sandbox Worker 账号领取并提交该 HIT 后，按回车继续...")
input()


👉 请在 https://workersandbox.mturk.com/ 用 Sandbox Worker 账号领取并提交该 HIT 后，按回车继续...


''

In [5]:
def list_assignments(hit_id: str):
    resp = mturk.list_assignments_for_hit(
        HITId=hit_id,
        AssignmentStatuses=['Submitted', 'Approved', 'Rejected'],
        MaxResults=100
    )
    return resp['Assignments']

def parse_external_answer(answer_xml: str) -> dict:
    root = ET.fromstring(answer_xml)
    result = {}
    for ans in root.findall('.//AnswerField'):
        qid = ans.findtext('QuestionIdentifier')
        text = ans.findtext('FreeText')
        result[qid] = text
    return result

assignments = list_assignments(hit_id)
if not assignments:
    print("⚠️ 尚未收到任何提交。")
else:
    for a in assignments:
        print("――― Assignment ID:", a['AssignmentId'])
        print("    Worker ID:    ", a['WorkerId'])
        print("    Answer：", parse_external_answer(a['Answer']))

⚠️ 尚未收到任何提交。


In [7]:
# attmept achiving this with agents

# publish_tool = PublishSurveyTool()
# fetch_tool   = FetchSurveyTool()

# deploy_agent  = Agent(config=deploy_cfg,  tools=[publish_tool])
# collect_agent = Agent(config=collect_cfg, tools=[fetch_tool])

# post_task = Task(
#     config=post_cfg,
#     agent=deploy_agent,
#     run=publish_tool.run
# )
# get_task = Task(
#     config=get_cfg,
#     agent=collect_agent,
#     run=fetch_tool.run
# )

# if __name__ == "__main__":
#     crew = Crew(
#         agents=[deploy_agent, collect_agent],
#         tasks =[post_task, get_task],
#         verbose=True
#     )

#     survey_definition = {
#         "title": "Customer Satisfaction Survey",
#         "fields": [
#             {
#                 "title": "How satisfied are you with our service?",
#                 "type": "multiple_choice",
#                 "properties": {
#                     "choices": [
#                         {"label": "Very Satisfied"},
#                         {"label": "Satisfied"},
#                         {"label": "Neutral"},
#                         {"label": "Dissatisfied"}
#                     ]
#                 }
#             }
#         ]
#     }

#     direct_pub = publish_tool.run(survey_definition)
#     print("Direct publish output:", direct_pub)
#     fid = direct_pub["id"]
#     direct_fetch = fetch_tool.run(fid)
#     print("Direct fetch output:", direct_fetch)

#     # print("\n=== Using Crew.kickoff ===")
#     # deploy_out  = crew.kickoff(inputs={"survey_definition": survey_definition})
#     # print("Crew deployed:", deploy_out)
#     # fid = deploy_out["id"]
#     # collect_out = crew.kickoff(inputs={"survey_id": fid})
#     # print("Crew collected:", collect_out)
